# Arduino Nano BLE33 Sense

This is a [micro-controller board](./ABX00031-datasheet.pdf) that has a few sensors on it. I will be specifically looking at 6 channels of the IMU, (3 acceleration channels and the 3 magnetic field channels). The acceleration channels will be used to figure the angle of the Optical Tube Assembly (OTA), commonly refereed to as the altitude (ALT).  The 3 magnetic field channels will be used to determine the direction the OTA is pointed, commonly refereed to as the azimuth (AZ). 

I want to see how accurate they are.  I do know that the ALT/AZ can be identified by the stars (image plate), time of day and location. If the IMU can be used to some degree we mat be able to get some basic functionality without having to process the image plates.

The BLE33 Sense can also be used to run a machine learning model. It may be feasible if the accuracy is good enough to determine the Declination the scope is pointed at and to do star tracking based on that.

## Sensor Output Format

The Arduino sketch outputs 8 tab-separated values:
- `aX, aY, aZ` - Accelerometer (g)
- `mX, mY, mZ` - Magnetometer (µT)
- `temperature` - Temperature (°C)
- `humidity` - Relative Humidity (%RH)

In [ ]:
# Install pyserial if needed
# !pdm add pyserial

## Upload Sketch to Arduino

Use `arduino-cli` to compile and upload the sketch. First, ensure arduino-cli is installed and configured.

In [ ]:
import subprocess
import shutil
from pathlib import Path

# Configuration
SKETCH_PATH = Path("../arduino/telescope_sensors/telescope_sensors.ino")
BOARD_FQBN = "arduino:mbed_nano:nano33ble"  # Fully Qualified Board Name
PORT = "/dev/ttyACM0"  # Adjust as needed

def check_arduino_cli():
    """Check if arduino-cli is installed and accessible."""
    if shutil.which("arduino-cli"):
        result = subprocess.run(["arduino-cli", "version"], capture_output=True, text=True)
        print(f"✓ arduino-cli found: {result.stdout.strip()}")
        return True
    else:
        print("✗ arduino-cli not found!")
        print("  Install with: curl -fsSL https://raw.githubusercontent.com/arduino/arduino-cli/master/install.sh | sh")
        return False

check_arduino_cli()

In [ ]:
# Install Arduino Nano 33 BLE board support (run once)
def install_board_support():
    """Install the Arduino Mbed OS Nano boards package."""
    print("Installing Arduino Nano 33 BLE board support...")
    
    # Update core index
    subprocess.run(["arduino-cli", "core", "update-index"], check=True)
    
    # Install mbed_nano core
    result = subprocess.run(
        ["arduino-cli", "core", "install", "arduino:mbed_nano"],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        print("✓ Board support installed")
    else:
        print(f"Installation output: {result.stdout}")
        if result.stderr:
            print(f"Errors: {result.stderr}")

# Uncomment to install:
# install_board_support()

In [ ]:
# Install required Arduino libraries (run once)
def install_libraries():
    """Install the required Arduino libraries for the sketch."""
    libraries = ["Arduino_LSM9DS1", "Arduino_HTS221"]
    
    for lib in libraries:
        print(f"Installing {lib}...")
        result = subprocess.run(
            ["arduino-cli", "lib", "install", lib],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            print(f"  ✓ {lib} installed")
        else:
            print(f"  ✗ Failed: {result.stderr}")

# Uncomment to install:
# install_libraries()

In [ ]:
def compile_sketch(sketch_path: Path = SKETCH_PATH, board: str = BOARD_FQBN):
    """Compile the Arduino sketch without uploading.
    
    Args:
        sketch_path: Path to the .ino file.
        board: Fully Qualified Board Name (FQBN).
        
    Returns:
        True if compilation successful, False otherwise.
    """
    sketch_dir = sketch_path.parent.resolve()
    print(f"Compiling {sketch_path.name}...")
    print(f"  Board: {board}")
    
    result = subprocess.run(
        ["arduino-cli", "compile", "--fqbn", board, str(sketch_dir)],
        capture_output=True, text=True
    )
    
    if result.returncode == 0:
        print("✓ Compilation successful!")
        # Extract binary size info
        for line in result.stdout.split('\n'):
            if 'Sketch uses' in line or 'Global variables' in line:
                print(f"  {line.strip()}")
        return True
    else:
        print("✗ Compilation failed!")
        print(result.stdout)
        print(result.stderr)
        return False

compile_sketch()

In [ ]:
def upload_sketch(sketch_path: Path = SKETCH_PATH, board: str = BOARD_FQBN, port: str = PORT):
    """Compile and upload the Arduino sketch to the board.
    
    Args:
        sketch_path: Path to the .ino file.
        board: Fully Qualified Board Name (FQBN).
        port: Serial port (e.g., /dev/ttyACM0).
        
    Returns:
        True if upload successful, False otherwise.
    """
    sketch_dir = sketch_path.parent.resolve()
    print(f"Uploading {sketch_path.name}...")
    print(f"  Board: {board}")
    print(f"  Port: {port}")
    
    result = subprocess.run(
        ["arduino-cli", "compile", "--upload", "--fqbn", board, "--port", port, str(sketch_dir)],
        capture_output=True, text=True
    )
    
    if result.returncode == 0:
        print("✓ Upload successful!")
        return True
    else:
        print("✗ Upload failed!")
        print(result.stdout)
        print(result.stderr)
        return False

# Upload the sketch (make sure Arduino is connected)
# upload_sketch()

In [ ]:
def list_boards():
    """List connected Arduino boards."""
    result = subprocess.run(
        ["arduino-cli", "board", "list"],
        capture_output=True, text=True
    )
    print("Connected boards:")
    print(result.stdout)
    return result.stdout

list_boards()

---
## Connect to Sensor

In [ ]:
import serial
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()

print("Available ports:", [port.device for port in ports])
for port in ports:
    print(f"  {port.device}: {port.description}")

In [ ]:
# Connect to Arduino - adjust port as needed
sensors = serial.Serial('/dev/ttyACM0', baudrate=115200, timeout=1)

In [ ]:
import threading

# Output format from Arduino:
# aX\taY\taZ\tmX\tmY\tmZ\ttemperature\thumidity

accelerometer = {}
magnetometer = {}
environment = {}

def read_sensors():
    """Read sensor data from Arduino Nano BLE33 Sense via serial.
    
    Parses 8 tab-separated values:
    - accelerometer: aX, aY, aZ (in g)
    - magnetometer: mX, mY, mZ (in µT)
    - temperature: °C
    - humidity: %RH
    """
    global accelerometer, magnetometer, environment

    while True:
        try:
            str_values = sensors.read_until(expected=b"\r\n").decode().strip().split("\t")
            if len(str_values) == 8:
                accelerometer = {
                    'aX': float(str_values[0]), 
                    'aY': float(str_values[1]), 
                    'aZ': float(str_values[2])
                }
                magnetometer = {
                    'mX': float(str_values[3]), 
                    'mY': float(str_values[4]), 
                    'mZ': float(str_values[5])
                }
                environment = {
                    'temperature': float(str_values[6]),
                    'humidity': float(str_values[7])
                }
            elif len(str_values) == 6:
                # Legacy 6-value format (no temp/humidity)
                accelerometer = {
                    'aX': float(str_values[0]), 
                    'aZ': float(str_values[1]), 
                    'aY': float(str_values[2])
                }
                magnetometer = {
                    'mX': float(str_values[3]), 
                    'mZ': float(str_values[4]), 
                    'mY': float(str_values[5])
                }
        except (ValueError, UnicodeDecodeError):
            pass  # Skip malformed lines

threading.Thread(target=read_sensors, daemon=True).start()

## Send Commands to Arduino

Send serial commands to control the sensor. Available commands:
- `RESET` - Reinitialize all sensors
- `STATUS` - Report sensor status and configuration  
- `CALIBRATE` - Run magnetometer calibration (rotate device slowly)
- `STOP` - Pause sensor output
- `START` - Resume sensor output

In [ ]:
def send_command(cmd: str, wait_response: bool = True, timeout: float = 2.0) -> str:
    """Send a command to the Arduino and optionally wait for response.
    
    Args:
        cmd: Command string (e.g., 'RESET', 'STATUS', 'CALIBRATE').
        wait_response: Whether to wait and collect response lines.
        timeout: Seconds to wait for response.
        
    Returns:
        Response string from Arduino.
    """
    import time
    
    # Clear any pending input
    sensors.reset_input_buffer()
    
    # Send command
    sensors.write(f"{cmd}\n".encode())
    print(f"Sent: {cmd}")
    
    if not wait_response:
        return ""
    
    # Collect response lines
    response_lines = []
    start_time = time.time()
    
    while (time.time() - start_time) < timeout:
        if sensors.in_waiting:
            line = sensors.readline().decode().strip()
            response_lines.append(line)
            print(line)
            # Check for end markers
            if line.startswith("===") and len(response_lines) > 1:
                break
            if line.startswith("OK:") or line.startswith("ERROR:"):
                break
        time.sleep(0.01)
    
    return "\n".join(response_lines)

In [ ]:
# Get sensor status
send_command("STATUS", timeout=3.0)

In [ ]:
# Reset/reinitialize sensors
send_command("RESET", timeout=5.0)

In [ ]:
# Run magnetometer calibration (rotate device slowly in all directions for ~10 seconds)
# This calculates hard-iron offsets to improve compass accuracy
send_command("CALIBRATE", timeout=15.0)

In [ ]:
# Stop sensor output (useful when sending multiple commands)
send_command("STOP")

In [ ]:
# Resume sensor output
send_command("START")

---
## Read Sensor Data

In [ ]:
# Display current sensor values
accelerometer, magnetometer, environment

In [ ]:
# Hold accelerometer snapshot for calculations
hold = accelerometer
hold

## Calculate the tilt with respect to _x_ (ALT)

From this [post](https://www.digikey.com/en/articles/using-an-accelerometer-for-inclination-sensing).

You need to do this because the accelerometer is sinusoidal in nature.

In [ ]:
import math

print(accelerometer)
tiltX = math.atan(accelerometer['aX'] / math.sqrt(
    (accelerometer['aY'] * accelerometer['aY']) + 
    (accelerometer['aZ'] * accelerometer['aZ'])
)) * (180 / math.pi)
tiltX

There is offset error due to a few things; like the non-perfect attachment of the sensor. It has a slight angle in the 3 axes. So I measured calculated angles when I knew the angle based on putting a level on the scope. I then used the 2 points to linearize the offset to be applied. Thus making the angle fit better.

In [ ]:
# Calibration values - measure these for your setup
calculated90 = 83.04141110222139
calculated0 = -1.7006050180649772

# y=mx+b where y=angle and x=calculated value
m = (90 - 0) / (calculated90 - calculated0)
b = 0 - (m * calculated0)

m, b

The following cell reports the Altitude for ALT/AZ so we can convert to RA/DEC

In [ ]:
print(accelerometer)
altitude = m * (math.atan(accelerometer['aX'] / math.sqrt(
    (accelerometer['aY'] * accelerometer['aY']) + 
    (accelerometer['aZ'] * accelerometer['aZ'])
)) * (180 / math.pi)) + b
altitude

## Determine Compass Heading (Azimuth)

https://www.w3.org/TR/magnetometer/

In [ ]:
def calibrate_heading(expected_heading: float) -> float:
    """Calculate heading offset based on known reference direction.
    
    Args:
        expected_heading: Known compass heading in degrees.
        
    Returns:
        Offset to apply to calculated headings.
    """
    calculated_heading = math.atan2(magnetometer['mY'], magnetometer['mX']) * (180 / math.pi)
    return expected_heading - calculated_heading

# Calibrate with a known heading (e.g., 280 degrees)
offset = calibrate_heading(280)

In [ ]:
# Calculate azimuth (compass heading)
azimuth = int((math.atan2(magnetometer['mZ'], magnetometer['mY']) * (180 / math.pi)) + offset)
azimuth

## Environment Data

Temperature and humidity from the HTS221 sensor.

In [ ]:
print(f"Temperature: {environment.get('temperature', 'N/A')}°C")
print(f"Humidity: {environment.get('humidity', 'N/A')}%RH")

## Coordinate Conversion

Convert between RA/DEC (equatorial) and ALT/AZ (horizontal) coordinate systems.
Requires observer location and current time.

In [ ]:
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time
from astropy import units as u
from datetime import datetime
from typing import NamedTuple

class ObserverLocation(NamedTuple):
    """Observer location on Earth."""
    latitude: float   # degrees, positive North
    longitude: float  # degrees, positive East
    elevation: float  # meters above sea level

# Set your observer location (example: Austin, TX)
OBSERVER_LOCATION = ObserverLocation(
    latitude=30.2672,
    longitude=-97.7431,
    elevation=150.0
)

def get_earth_location(loc: ObserverLocation = OBSERVER_LOCATION) -> EarthLocation:
    """Get astropy EarthLocation from observer location.
    
    Args:
        loc: Observer location tuple.
        
    Returns:
        Astropy EarthLocation object.
    """
    return EarthLocation(
        lat=loc.latitude * u.deg,
        lon=loc.longitude * u.deg,
        height=loc.elevation * u.m
    )

print(f"Observer location: {OBSERVER_LOCATION.latitude:.4f}°N, {OBSERVER_LOCATION.longitude:.4f}°E")

In [ ]:
def radec_to_altaz(
    ra: float, 
    dec: float, 
    obstime: datetime | None = None,
    location: ObserverLocation = OBSERVER_LOCATION
) -> tuple[float, float]:
    """Convert RA/DEC to ALT/AZ for observer location and time.
    
    Args:
        ra: Right Ascension in degrees (0-360) or hours (0-24 if < 24).
        dec: Declination in degrees (-90 to +90).
        obstime: Observation time (UTC). Defaults to now.
        location: Observer location on Earth.
        
    Returns:
        Tuple of (altitude, azimuth) in degrees.
        
    Example:
        >>> alt, az = radec_to_altaz(83.633, 22.014)  # M42 (Orion Nebula)
        >>> print(f"ALT: {alt:.2f}°, AZ: {az:.2f}°")
    """
    # Handle RA in hours vs degrees
    if ra < 24:
        ra = ra * 15  # Convert hours to degrees
    
    # Default to current time
    if obstime is None:
        obstime = datetime.utcnow()
    
    # Create coordinate objects
    earth_loc = get_earth_location(location)
    obs_time = Time(obstime)
    altaz_frame = AltAz(obstime=obs_time, location=earth_loc)
    
    # Transform
    sky_coord = SkyCoord(ra=ra * u.deg, dec=dec * u.deg, frame='icrs')
    altaz_coord = sky_coord.transform_to(altaz_frame)
    
    return altaz_coord.alt.deg, altaz_coord.az.deg


def altaz_to_radec(
    alt: float, 
    az: float, 
    obstime: datetime | None = None,
    location: ObserverLocation = OBSERVER_LOCATION
) -> tuple[float, float]:
    """Convert ALT/AZ to RA/DEC for observer location and time.
    
    Args:
        alt: Altitude in degrees (0 = horizon, 90 = zenith).
        az: Azimuth in degrees (0 = North, 90 = East).
        obstime: Observation time (UTC). Defaults to now.
        location: Observer location on Earth.
        
    Returns:
        Tuple of (ra, dec) in degrees.
        
    Example:
        >>> ra, dec = altaz_to_radec(45.0, 180.0)  # South, 45° up
        >>> print(f"RA: {ra:.4f}°, DEC: {dec:.4f}°")
    """
    # Default to current time
    if obstime is None:
        obstime = datetime.utcnow()
    
    # Create coordinate objects
    earth_loc = get_earth_location(location)
    obs_time = Time(obstime)
    altaz_frame = AltAz(obstime=obs_time, location=earth_loc)
    
    # Transform
    altaz_coord = SkyCoord(alt=alt * u.deg, az=az * u.deg, frame=altaz_frame)
    icrs_coord = altaz_coord.transform_to('icrs')
    
    return icrs_coord.ra.deg, icrs_coord.dec.deg


# Test the converters
print("Testing coordinate conversion...")
test_ra, test_dec = 83.633, 22.014  # M42 Orion Nebula
alt, az = radec_to_altaz(test_ra, test_dec)
print(f"M42 (RA={test_ra}°, DEC={test_dec}°) → ALT={alt:.2f}°, AZ={az:.2f}°")

# Round-trip test
ra_back, dec_back = altaz_to_radec(alt, az)
print(f"Round-trip: RA={ra_back:.4f}°, DEC={dec_back:.4f}° (diff: {abs(test_ra-ra_back):.6f}°)")

## Sensor Calibration

The `SensorCalibration` class maps raw sensor ALT/AZ to corrected values using plate-solved camera images as ground truth.

**Workflow:**
1. Read sensor ALT/AZ at current position
2. Take image and plate-solve for true RA/DEC
3. Convert RA/DEC to true ALT/AZ
4. Add calibration point (sensor vs true)
5. After multiple points, compute transform
6. Apply transform to future sensor readings

In [ ]:
import numpy as np
from dataclasses import dataclass, field
from typing import Optional
import json

@dataclass
class CalibrationPoint:
    """A single calibration measurement."""
    sensor_alt: float      # Sensor-reported altitude
    sensor_az: float       # Sensor-reported azimuth
    true_alt: float        # True altitude (from plate solve)
    true_az: float         # True azimuth (from plate solve)
    timestamp: datetime    # When measurement was taken
    ra: Optional[float] = None   # Original RA from plate solve
    dec: Optional[float] = None  # Original DEC from plate solve


@dataclass
class SensorTransform:
    """Transform parameters to correct sensor readings."""
    alt_scale: float = 1.0       # Altitude scale factor
    alt_offset: float = 0.0      # Altitude offset (degrees)
    az_scale: float = 1.0        # Azimuth scale factor  
    az_offset: float = 0.0       # Azimuth offset (degrees, includes magnetic declination)
    
    def apply(self, sensor_alt: float, sensor_az: float) -> tuple[float, float]:
        """Apply transform to sensor readings.
        
        Args:
            sensor_alt: Raw sensor altitude.
            sensor_az: Raw sensor azimuth.
            
        Returns:
            Tuple of (corrected_alt, corrected_az).
        """
        corrected_alt = self.alt_scale * sensor_alt + self.alt_offset
        corrected_az = (self.az_scale * sensor_az + self.az_offset) % 360
        return corrected_alt, corrected_az
    
    def to_dict(self) -> dict:
        """Export transform as dictionary."""
        return {
            'alt_scale': self.alt_scale,
            'alt_offset': self.alt_offset,
            'az_scale': self.az_scale,
            'az_offset': self.az_offset
        }
    
    @classmethod
    def from_dict(cls, data: dict) -> 'SensorTransform':
        """Create transform from dictionary."""
        return cls(**data)


class SensorCalibration:
    """Calibrate IMU sensor against plate-solved camera images.
    
    Collects calibration points (sensor vs true ALT/AZ) and computes
    a linear transform to correct future sensor readings.
    
    Example:
        >>> cal = SensorCalibration()
        >>> # Add points from plate solves
        >>> cal.add_point_from_radec(sensor_alt=45.2, sensor_az=182.5, 
        ...                          ra=83.633, dec=22.014)
        >>> cal.add_point_from_radec(sensor_alt=62.1, sensor_az=95.3,
        ...                          ra=201.298, dec=-43.019)
        >>> # Compute and apply transform
        >>> cal.compute_transform()
        >>> corrected_alt, corrected_az = cal.apply(45.0, 180.0)
    """
    
    def __init__(self, location: ObserverLocation = OBSERVER_LOCATION):
        """Initialize calibration with observer location.
        
        Args:
            location: Observer location for coordinate conversion.
        """
        self.location = location
        self.points: list[CalibrationPoint] = []
        self.transform: Optional[SensorTransform] = None
    
    def add_point(
        self, 
        sensor_alt: float, 
        sensor_az: float,
        true_alt: float, 
        true_az: float,
        timestamp: Optional[datetime] = None
    ) -> None:
        """Add a calibration point with known true ALT/AZ.
        
        Args:
            sensor_alt: Altitude from sensor.
            sensor_az: Azimuth from sensor.
            true_alt: True altitude (from plate solve).
            true_az: True azimuth (from plate solve).
            timestamp: Measurement time. Defaults to now.
        """
        if timestamp is None:
            timestamp = datetime.utcnow()
        
        point = CalibrationPoint(
            sensor_alt=sensor_alt,
            sensor_az=sensor_az,
            true_alt=true_alt,
            true_az=true_az,
            timestamp=timestamp
        )
        self.points.append(point)
        print(f"Added calibration point {len(self.points)}: "
              f"sensor({sensor_alt:.2f}°, {sensor_az:.2f}°) → "
              f"true({true_alt:.2f}°, {true_az:.2f}°)")
    
    def add_point_from_radec(
        self,
        sensor_alt: float,
        sensor_az: float,
        ra: float,
        dec: float,
        timestamp: Optional[datetime] = None
    ) -> None:
        """Add a calibration point from plate-solved RA/DEC.
        
        Args:
            sensor_alt: Altitude from sensor.
            sensor_az: Azimuth from sensor.
            ra: Right Ascension from plate solve (degrees).
            dec: Declination from plate solve (degrees).
            timestamp: Measurement time. Defaults to now.
        """
        if timestamp is None:
            timestamp = datetime.utcnow()
        
        # Convert RA/DEC to ALT/AZ
        true_alt, true_az = radec_to_altaz(ra, dec, timestamp, self.location)
        
        point = CalibrationPoint(
            sensor_alt=sensor_alt,
            sensor_az=sensor_az,
            true_alt=true_alt,
            true_az=true_az,
            timestamp=timestamp,
            ra=ra,
            dec=dec
        )
        self.points.append(point)
        print(f"Added calibration point {len(self.points)}: "
              f"sensor({sensor_alt:.2f}°, {sensor_az:.2f}°) → "
              f"RA/DEC({ra:.4f}°, {dec:.4f}°) → "
              f"true ALT/AZ({true_alt:.2f}°, {true_az:.2f}°)")
    
    def compute_transform(self, min_points: int = 2) -> SensorTransform:
        """Compute transform from calibration points.
        
        Uses linear least squares to fit:
        true_alt = alt_scale * sensor_alt + alt_offset
        true_az  = az_scale  * sensor_az  + az_offset
        
        Args:
            min_points: Minimum points required.
            
        Returns:
            Computed SensorTransform.
            
        Raises:
            ValueError: If insufficient calibration points.
        """
        if len(self.points) < min_points:
            raise ValueError(f"Need at least {min_points} calibration points, "
                           f"have {len(self.points)}")
        
        # Extract arrays
        sensor_alt = np.array([p.sensor_alt for p in self.points])
        sensor_az = np.array([p.sensor_az for p in self.points])
        true_alt = np.array([p.true_alt for p in self.points])
        true_az = np.array([p.true_az for p in self.points])
        
        # Handle azimuth wrap-around (find minimal angular difference)
        az_diff = true_az - sensor_az
        az_diff = (az_diff + 180) % 360 - 180  # Normalize to -180..180
        
        # Linear fit for altitude: true = scale * sensor + offset
        if len(self.points) >= 2:
            alt_fit = np.polyfit(sensor_alt, true_alt, 1)
            alt_scale, alt_offset = alt_fit[0], alt_fit[1]
        else:
            alt_scale = 1.0
            alt_offset = np.mean(true_alt - sensor_alt)
        
        # For azimuth, primarily fit offset (scale usually ~1.0)
        if len(self.points) >= 2:
            az_fit = np.polyfit(sensor_az, true_az, 1)
            az_scale, az_offset = az_fit[0], az_fit[1]
        else:
            az_scale = 1.0
            az_offset = np.mean(az_diff)
        
        self.transform = SensorTransform(
            alt_scale=alt_scale,
            alt_offset=alt_offset,
            az_scale=az_scale,
            az_offset=az_offset
        )
        
        # Report results
        print("Transform computed:")
        print(f"  ALT: corrected = {alt_scale:.4f} × sensor + {alt_offset:.2f}°")
        print(f"  AZ:  corrected = {az_scale:.4f} × sensor + {az_offset:.2f}°")
        
        # Report residuals
        pred_alt = alt_scale * sensor_alt + alt_offset
        pred_az = (az_scale * sensor_az + az_offset) % 360
        alt_residual = np.std(true_alt - pred_alt)
        az_residual = np.std((true_az - pred_az + 180) % 360 - 180)
        print(f"  Residuals: ALT σ={alt_residual:.3f}°, AZ σ={az_residual:.3f}°")
        
        return self.transform
    
    def apply(self, sensor_alt: float, sensor_az: float) -> tuple[float, float]:
        """Apply transform to get corrected ALT/AZ.
        
        Args:
            sensor_alt: Raw sensor altitude.
            sensor_az: Raw sensor azimuth.
            
        Returns:
            Tuple of (corrected_alt, corrected_az).
            
        Raises:
            ValueError: If transform not computed yet.
        """
        if self.transform is None:
            raise ValueError("Transform not computed. Call compute_transform() first.")
        return self.transform.apply(sensor_alt, sensor_az)
    
    def get_current_sensor_altaz(self) -> tuple[float, float]:
        """Get current ALT/AZ from sensor (using global accelerometer/magnetometer).
        
        Returns:
            Tuple of (altitude, azimuth) from sensor.
        """
        # Calculate altitude from accelerometer
        ax = accelerometer.get('aX', 0)
        ay = accelerometer.get('aY', 0)
        az_accel = accelerometer.get('aZ', 0)
        
        sensor_alt = math.atan(ax / math.sqrt(ay*ay + az_accel*az_accel)) * (180 / math.pi)
        
        # Calculate azimuth from magnetometer
        mx = magnetometer.get('mX', 0)
        my = magnetometer.get('mY', 0)
        mz = magnetometer.get('mZ', 0)
        
        sensor_az = math.atan2(my, mx) * (180 / math.pi)
        if sensor_az < 0:
            sensor_az += 360
        
        return sensor_alt, sensor_az
    
    def save(self, filepath: str) -> None:
        """Save calibration to JSON file.
        
        Args:
            filepath: Path to save calibration data.
        """
        data = {
            'location': {
                'latitude': self.location.latitude,
                'longitude': self.location.longitude,
                'elevation': self.location.elevation
            },
            'points': [
                {
                    'sensor_alt': p.sensor_alt,
                    'sensor_az': p.sensor_az,
                    'true_alt': p.true_alt,
                    'true_az': p.true_az,
                    'timestamp': p.timestamp.isoformat(),
                    'ra': p.ra,
                    'dec': p.dec
                }
                for p in self.points
            ],
            'transform': self.transform.to_dict() if self.transform else None
        }
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)
        print(f"Calibration saved to {filepath}")
    
    @classmethod
    def load(cls, filepath: str) -> 'SensorCalibration':
        """Load calibration from JSON file.
        
        Args:
            filepath: Path to calibration data.
            
        Returns:
            Loaded SensorCalibration instance.
        """
        with open(filepath, 'r') as f:
            data = json.load(f)
        
        loc = ObserverLocation(**data['location'])
        cal = cls(location=loc)
        
        for p in data['points']:
            point = CalibrationPoint(
                sensor_alt=p['sensor_alt'],
                sensor_az=p['sensor_az'],
                true_alt=p['true_alt'],
                true_az=p['true_az'],
                timestamp=datetime.fromisoformat(p['timestamp']),
                ra=p.get('ra'),
                dec=p.get('dec')
            )
            cal.points.append(point)
        
        if data.get('transform'):
            cal.transform = SensorTransform.from_dict(data['transform'])
        
        print(f"Loaded calibration with {len(cal.points)} points from {filepath}")
        return cal


# Create calibration instance
calibration = SensorCalibration(OBSERVER_LOCATION)
print(f"SensorCalibration initialized for {OBSERVER_LOCATION.latitude:.4f}°N")

### Calibration Workflow

Run through these cells to calibrate the sensor against plate-solved images.

In [ ]:
# Step 1: Get current sensor reading
sensor_alt, sensor_az = calibration.get_current_sensor_altaz()
print(f"Current sensor reading: ALT={sensor_alt:.2f}°, AZ={sensor_az:.2f}°")

In [ ]:
# Step 2: After plate-solving image, add calibration point
# Replace these values with your plate-solve results

# Example: M42 Orion Nebula plate solve result
plate_solve_ra = 83.820  # degrees (or hours if < 24)
plate_solve_dec = -5.391  # degrees

# Add the calibration point (uses current sensor reading and plate solve result)
calibration.add_point_from_radec(
    sensor_alt=sensor_alt,
    sensor_az=sensor_az,
    ra=plate_solve_ra,
    dec=plate_solve_dec
)

In [ ]:
# Step 3: Repeat steps 1-2 for multiple positions across the sky
# More points = better transform, especially at different altitudes

# View current calibration points
print(f"Calibration points: {len(calibration.points)}")
for i, p in enumerate(calibration.points, 1):
    print(f"  {i}: sensor({p.sensor_alt:.1f}°, {p.sensor_az:.1f}°) → true({p.true_alt:.1f}°, {p.true_az:.1f}°)")

In [ ]:
# Step 4: Compute the transform (need at least 2 points)
if len(calibration.points) >= 2:
    calibration.compute_transform()
else:
    print(f"Need at least 2 calibration points, currently have {len(calibration.points)}")

In [ ]:
# Step 5: Apply transform to get corrected position
sensor_alt, sensor_az = calibration.get_current_sensor_altaz()

if calibration.transform:
    corrected_alt, corrected_az = calibration.apply(sensor_alt, sensor_az)
    print(f"Raw sensor:  ALT={sensor_alt:.2f}°, AZ={sensor_az:.2f}°")
    print(f"Corrected:   ALT={corrected_alt:.2f}°, AZ={corrected_az:.2f}°")
    
    # Convert to RA/DEC for display
    ra, dec = altaz_to_radec(corrected_alt, corrected_az)
    print(f"Estimated:   RA={ra:.4f}° ({ra/15:.4f}h), DEC={dec:.4f}°")
else:
    print("Transform not computed yet. Run Step 4 first.")

In [ ]:
# Save/Load calibration for later sessions
CALIBRATION_FILE = "../data/sensor_calibration.json"

# Save current calibration
# calibration.save(CALIBRATION_FILE)

# Load existing calibration
# calibration = SensorCalibration.load(CALIBRATION_FILE)